In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [2]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [3]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Prepare Script

In [4]:
%%writefile ./src/prepare.py

from azureml.core import Run
from argparse import ArgumentParser
import numpy as np
import pandas as pd
import datetime
import os

run = Run.get_context()

parser = ArgumentParser()
parser.add_argument('--output', dest='prepared_data')
args = parser.parse_args()

# load datasets
df_symptoms = run.input_datasets['symptomcodes'].to_pandas_dataframe()
df = run.input_datasets['df_raw'].to_pandas_dataframe()

##########################################################################################

# get only data from last t years
t = 3
df = df[df['Job Card.Date Start Work']>(datetime.datetime.today() - datetime.timedelta(days=t*365))]

##########################################################################################

# clean data
df = df.replace(['', '0', '-', '000','N/A'], np.nan)
df = df.dropna().reset_index(drop=True)

##########################################################################################

# combine Component/Failure Code in train data
df = pd.concat([df, pd.DataFrame(df.apply(lambda x: (x['Job Card.ComponentCode'],x['Job Card.FailureCode']), axis=1), columns=['CompFail'])], axis=1)

# combine Component/Failure Code in symptom table
df_symptoms = df_symptoms[['ComponentCode', 'FailureCode', 'Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']]
df_symptoms = pd.concat([df_symptoms, pd.DataFrame(df_symptoms.apply(lambda x: (x['ComponentCode'],x['FailureCode']),axis=1), columns=['CompFail'])],axis=1)

# merge train data on symptoms
df = pd.merge(df, df_symptoms, on='CompFail', how='left')
df = pd.concat([df, pd.DataFrame(df[['Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']].apply(lambda x: tuple([ x[col] for col in ['Symptom1','Symptom2','Symptom3','Symptom4'] if str(x[col]) != 'None' ]), axis=1), columns=['Symptoms'])], axis=1)

# remove rows with no symptoms
df = df[df['Symptoms']!=()]

##########################################################################################

# merge into one row per case
df = df.groupby('Job Card.JobCard Number').apply(lambda x: pd.Series({
    'ProductId': ' '.join(x['Installed Base.InstalledBase ProductID'].unique()),
    'Country': ' '.join(x['Location.Country'].unique()),
    'Symptoms': ' '.join(map(str, list(set(x['Symptoms'].sum())))),
    'ProductNrs': ' '.join(x['Product.Product Number'].unique()),
    'Start': x['Job Card.Date Start Work'].min(),
    'End': x['Job Card.Date End Work'].max()
  })).reset_index()

#########################################################################################

# split data (test data from last t_test years)
t_test = 0.5
df_train = df[df['Start']<(datetime.datetime.today() - datetime.timedelta(days=t_test*365))]
df_test = df[df['Start']>=(datetime.datetime.today() - datetime.timedelta(days=t_test*365))]

# save train and test data
path = arg.prepared_data if args.prepared_data else './outputs'
os.makedirs(path, exist_ok=True)
df_train.to_csv(path + '/train_data.csv', sep=';', header=True, index=False)
df_test.to_csv(path + '/test_data.csv', sep=';', header=True, index=False)

run.complete()

Overwriting ./src/prepare.py


## Create Estimator

In [5]:
est = Estimator(entry_script='prepare.py', source_directory='src', 
              inputs=[   ws.datasets[cfg['symptomcodes_dataset']].as_named_input('symptomcodes'), 
                         ws.datasets[cfg['raw_data_dataset']].as_named_input('df_raw')       ],
              compute_target='local', environment_definition=ws.environments[cfg['env_name']])

## Run Experiment

In [6]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: LookUpPrediction_1592829616_abf28b61
Web View: https://ml.azure.com/experiments/LookUpPrediction/runs/LookUpPrediction_1592829616_abf28b61?wsid=/subscriptions/793146d9-d4dc-4a73-9728-76c4ffd0cc0d/resourcegroups/rg_dynamics_test/workspaces/resdynml1test

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-06-22T12:40:18.183462
Starting the daemon thread to refresh tokens in background for process with pid = 8
Entering Run History Context Manager.
Preparing to call script [ prepare.py ] with arguments: []
After variable expansion, calling script [ prepare.py ] with arguments: []

Starting the daemon thread to refresh tokens in background for process with pid = 8


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service.
Cleaning up all outstanding Run operations, waiting 300.0 seconds
2 items cleaning up...
Cleanup took 0.17834973335266113 seconds

Execution Summary
RunId: LookUpPre

{'runId': 'LookUpPrediction_1592829616_abf28b61',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-06-22T12:40:17.517455Z',
 'endTimeUtc': '2020-06-22T12:45:03.819862Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'a602af5d-f404-4baa-898a-1ff229aff30f'},
 'inputDatasets': [{'dataset': {'id': '02e6cb83-4d0c-42b2-bbef-e103c74b3a3c'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'df_raw', 'mechanism': 'Direct'}}, {'dataset': {'id': '88af5740-1a1b-4e09-8129-d3c538680909'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'symptomcodes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'prepare.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'df_raw': {'dataLocation': {'dataset': {'id': '02e6cb83-4d0c-42b2-bbef-e103c74b3a3c',
      'name': None,
      'version': None},
  

## Register Datasets

In [7]:
run.download_file('outputs/train_data.csv', output_file_path='artifacts/train_data.csv')
ds = ws.datastores[cfg['storage']]
data_ref = ds.upload_files(['artifacts/train_data.csv'], target_path='./'+cfg['experiment_name'], overwrite=True)
prepared_data_dataset = Dataset.Tabular.from_delimited_files(data_ref, separator=';', header=True, infer_column_types=True)
prepared_data_dataset.register(ws, cfg['train_dataset'], create_new_version=True)

Uploading an estimated of 1 files
Uploading artifacts/train_data.csv
Uploaded artifacts/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', './LookUpPrediction')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "7b05e6a0-bdcb-4dac-badd-43da21ebfb81",
    "name": "LookUpTrainData",
    "version": 1,
    "workspace": "Workspace.create(name='resdynml1test', subscription_id='793146d9-d4dc-4a73-9728-76c4ffd0cc0d', resource_group='rg_dynamics_test')"
  }
}

In [8]:
run.download_file('outputs/test_data.csv', output_file_path='artifacts/test_data.csv')
ds = ws.datastores[cfg['storage']]
data_ref = ds.upload_files(['artifacts/test_data.csv'], target_path='./'+cfg['experiment_name'], overwrite=True)
prepared_data_dataset = Dataset.Tabular.from_delimited_files(data_ref, separator=';', header=True, infer_column_types=False)
prepared_data_dataset.register(ws, cfg['test_dataset'], create_new_version=True)

Uploading an estimated of 1 files
Uploading artifacts/test_data.csv
Uploaded artifacts/test_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', './LookUpPrediction')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns"
  ],
  "registration": {
    "id": "1cf29584-c0d6-4a92-b14d-3bc8bcd83723",
    "name": "LookUpTestData",
    "version": 1,
    "workspace": "Workspace.create(name='resdynml1test', subscription_id='793146d9-d4dc-4a73-9728-76c4ffd0cc0d', resource_group='rg_dynamics_test')"
  }
}